In [9]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import null_space
from math import floor, ceil, sqrt
import sys

In [10]:
# # Setup
# Hyper-parameters

class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

FLAGS = AttrDict(
    K = 1000, # number of arms
    d = 10, # dimension
    T = 100, # Number of time steps
    maxT = 10000,
    M = 100, # number of agents
    tol = 0.5, # finds a (1+tol)-approx coreset
    maxiter = 1000, 
)


def get_reward(x):
    return Theta@x+np.random.normal()

class Agent:
    def __init__(self,X):
        K,d = X.shape
        self.X = X
        self.remaining = set(np.arange(K))
        self.A = X
    def find_coreset(self):
        uu = coreset(self.A)
        full = np.arange(FLAGS.K)
        self.support = full[uu>0]
    def update(self, theta, phase):
        eps = 2.0**(-phase+1)
        valvec = self.A@theta
        maxval = np.max(self.A@theta)
        remaining = self.remaining.copy()
        for a in remaining:
            if X[a,:]@theta<maxval-eps:
                self.remaining.remove(a)
        self.A = X[list(self.remaining),:]
    def return_result(self):
        return self.results
    def receive_instruction(self,inst):
        # Instruction is a dictionary: {index: times}
        self.inst = inst
        self.counts = {}
        self.results = {}
        for index in inst:
            self.counts[index]=0
            self.results[index]=0
    def step(self):
        inst = self.inst
        for idx in inst:
            if self.counts[idx]<inst[idx]:
                y = get_reward(self.X[self.support[idx],:])
                self.results[idx] += y
                self.counts[idx] += 1
                return self.X[self.support[idx],:]
        return self.A[0,:]


# In[8]:


# Usage: X is K-by-d matrix
def coreset(X):
    # Initilization
    n = FLAGS.d
    K = FLAGS.K
    c = np.ones(n)
    j=1
    ys = []
    u_ky = np.zeros(K)
    for j in np.arange(n):
        vals = X@c
        # Find argmax
        argmax = np.argmax(vals)
        argmin = np.argmin(vals)
        u_ky[argmin] += 1/(2*n)
        u_ky[argmax] += 1/(2*n)
        y = X[argmax,:]-X[argmin,:]
        ys.append(y)
        Y = np.column_stack(ys)
        if j<(n-1):
            H=null_space(Y.T)
            c = H[:,0]
    # Iterate
    u = u_ky
    d = FLAGS.d
    tol = FLAGS.tol
    max_iter = FLAGS.maxiter
    for i in range(max_iter):
        Vinv = np.linalg.inv(V(u,X))
        # find argmax
        vals = np.diag(X@Vinv@X.T)
        tpmax = np.max(vals)
        argmax = np.argmax(vals)
        if tpmax<(d*(1+tol)):
            break
        gamma = np.squeeze((tpmax/d-1)/(tpmax-1))
        e = np.zeros(K)
        e[argmax] = 1.0
        u = u*(1-gamma)+gamma*e
    return u

def V(u,X):
    # X is K-by-d
    K,d = X.shape
    mat = np.zeros((d,d))
    for i in range(K):
        vec = X[i,:].reshape([d,1])
        mat += u[i] * vec @ vec.T
    return mat

In [ ]:
# Instance-creation
np.random.seed(187)
counter = 0
tracex = [] # used for drawing pictures
tracey = [] # used for drawing pictures
CommCost = 0

while (FLAGS.T <= FLAGS.maxT) :
    print(FLAGS.T)
    Regret = 0

    for experi in range(5) : # 5 instances per T
        Theta = np.random.normal(size=FLAGS.d)
        #Theta = Theta / np.linalg.norm(Theta)
        rho = 2 / sqrt(FLAGS.T)
        X=np.random.random(size=(FLAGS.K, FLAGS.d))*rho - np.ones((FLAGS.K, FLAGS.d))*0.5*rho
        rstar = np.max(X@Theta)
        best = np.argmax(X@Theta)

        counter += 1

        #print(Theta)
        #print(X)

        # # Run

        # DELB

        M = FLAGS.M
        agents = [Agent(X) for i in range(M)]
        time_cnt = 0
        max_phase = np.floor(np.log(FLAGS.T)/np.log(4))
        A = X
        remaining_arms = set(np.arange(FLAGS.K))
        T = FLAGS.T
        d = FLAGS.d
        C_1 = 0.01 # Needs to be tuned
        print('MAX_PHASE %d' % max_phase)
        for phase in np.arange(max_phase+2):
            print('PHASE %d' % phase)
            if len(remaining_arms)<1.5*d:
                u = np.zeros(FLAGS.K)
                for i in remaining_arms:
                    u[i] = 1.0/len(remaining_arms)
            else:
                u = coreset(A)

            full = np.arange(FLAGS.K)
            support = full[u>0]
            #print(support)
            for i in range(M):
                if len(remaining_arms)<1.5*d:
                    agents[i].support = support
                else:
                    agents[i].find_coreset()
            scheme = {}
            Mphase = 4.0**phase * C_1 * d * d * np.log(M*T)
            for idx in range(len(support)):
                scheme[support[idx]] = ceil(u[support[idx]]*Mphase)
            total = sum(scheme[v] for v in scheme)
            if total%M != 0:
                b = M-total%M
                scheme[support[-1]] += b
                total += b
            phase_len = total//M
            # Send Instruction to agent
            remaining = scheme.copy()
            pointer = 0
            instructions = [{} for i in range(M)]
            for i in range(M):
                space = phase_len
                while pointer<len(support) and remaining[support[pointer]]<=space:
                    instructions[i][pointer] = remaining[support[pointer]]
                    space -= remaining[support[pointer]]
                    pointer+=1
                if space>0:
                    instructions[i][pointer] = space
                    remaining[support[pointer]] -= space
            #print(instructions)
            for i in range(M):
                # TODO: count bits
                # CommCost += sys.getsizeof(instructions[i])
                CommCost += len(instructions[i])
                agents[i].receive_instruction(instructions[i])
            for t in range(phase_len):
                time_cnt += 1
                #print(time_cnt)
                for i in range(M):
                    act = agents[i].step()
                    Regret += rstar-Theta@act
                    #TODO: count regret here
    #            tracey.append(Regret)
                if (time_cnt>=FLAGS.T):
                    break
            if (time_cnt>=FLAGS.T):
                break
            results = {}
            for v in scheme:
                results[v] = 0
            for i in range(M):
                # TODO: count bits here
                res = agents[i].return_result()
                # CommCost += sys.getsizeof(res)
                CommCost += len(res)
                for v in res:
                    results[support[v]] += res[v]
            # Perform regression
            y = np.zeros(d)
            Vmat = np.zeros((d,d))
            for v in scheme:
                vec = X[v,:].reshape([d,1])
                y += X[v,:]*results[v]
                Vmat += vec@vec.T*scheme[v]
            theta_hat = np.linalg.pinv(Vmat)@y
            print(theta_hat)
            # Perform elimination
            for i in range(M):
                # CommCost += sys.getsizeof(theta_hat)
                CommCost += theta_hat.size
                agents[i].update(theta_hat, phase)

            eps = 2.0**(-phase+1)
            maxval = np.max(A@theta_hat)
            tmpremaining = remaining_arms.copy()
            for a in tmpremaining:
                if X[a,:]@theta_hat<maxval-eps:
                    remaining_arms.remove(a)
            A = X[list(remaining_arms),:]
            print(remaining_arms)
    tracex.append(FLAGS.T)
    tracey.append(Regret / 5)
    #print(Regret)
    FLAGS.T += 200

CommCost = CommCost * 8 / counter
print(CommCost)
File = open('DELB.txt', 'w')
File.write(str(CommCost))
File.write('\n')
for k in range(len(tracex)) :
    File.write(str(tracex[k]) + str(' ') + str(tracey[k]))
    File.write('\n')
File.close()
plt.plot(tracex, tracey, label = 'DELB')
plt.xlabel('Time')
plt.ylabel('Cumulative Regret')
plt.legend()
plt.savefig('DELB.png')
Finish = open('DELB_finish.txt', 'w')
Finish.close()

100
MAX_PHASE 3
PHASE 0
[ 0.56466362 -1.79098779  5.62195433  0.13276538  1.49515357  3.23701292
  2.02203149 -1.75119381 -1.24056893 -1.8787734 ]
{1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 118, 119, 120, 121, 122, 123, 125, 126, 127, 128, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 

[-3.71377489 -0.07080625  2.33008383 -3.52448366  0.72837313 -0.77152792
 -6.3607097   0.41946386 -1.94363171  3.21347616]
{1, 9, 13, 14, 19, 21, 23, 24, 27, 31, 39, 44, 49, 50, 53, 55, 58, 65, 69, 77, 78, 79, 98, 100, 112, 117, 119, 141, 149, 161, 167, 171, 181, 188, 200, 203, 205, 206, 210, 213, 214, 216, 219, 221, 223, 225, 235, 247, 252, 256, 258, 259, 262, 267, 269, 272, 275, 277, 289, 296, 298, 302, 303, 305, 309, 311, 312, 313, 317, 318, 320, 322, 324, 332, 339, 343, 346, 356, 357, 359, 360, 363, 371, 379, 380, 383, 385, 387, 388, 397, 420, 425, 429, 443, 445, 447, 458, 459, 460, 463, 472, 473, 476, 501, 502, 503, 507, 515, 518, 523, 524, 528, 532, 534, 539, 542, 560, 575, 576, 577, 591, 593, 596, 598, 602, 604, 606, 610, 613, 617, 620, 628, 633, 636, 639, 641, 647, 649, 655, 656, 660, 666, 668, 670, 671, 677, 680, 693, 699, 706, 711, 715, 717, 719, 720, 721, 722, 727, 728, 729, 733, 735, 744, 745, 747, 748, 749, 751, 756, 760, 768, 781, 782, 783, 785, 790, 794, 800, 806, 809, 8

[-0.92151241 -3.52858463  2.28808187  0.67777163 -5.60511311 -3.01731895
 -0.9954924  -2.57740609  2.10777281  0.0454457 ]
{2, 3, 5, 6, 9, 13, 14, 22, 29, 40, 44, 46, 48, 52, 56, 60, 62, 88, 91, 94, 95, 103, 105, 108, 115, 125, 128, 134, 136, 137, 145, 148, 154, 156, 176, 179, 185, 186, 190, 195, 196, 204, 205, 207, 208, 210, 214, 216, 220, 226, 234, 242, 246, 249, 251, 266, 267, 268, 269, 277, 280, 291, 292, 293, 294, 299, 305, 307, 308, 309, 324, 332, 338, 339, 346, 351, 352, 359, 360, 362, 370, 372, 377, 385, 392, 397, 398, 399, 401, 402, 403, 406, 408, 409, 411, 423, 424, 427, 434, 444, 454, 458, 461, 471, 473, 478, 480, 481, 492, 497, 503, 504, 510, 513, 514, 515, 529, 536, 548, 552, 570, 572, 576, 579, 582, 591, 594, 598, 599, 602, 611, 615, 616, 618, 619, 622, 632, 635, 636, 637, 639, 643, 645, 646, 647, 664, 672, 673, 679, 696, 698, 699, 715, 718, 720, 735, 736, 745, 749, 751, 754, 756, 764, 768, 769, 773, 774, 776, 778, 779, 781, 797, 798, 801, 803, 804, 807, 809, 816, 818, 82

[ 2.29070358  0.61849837  2.27919846 -2.50099516  1.98706342 -1.79285696
  1.93486655  0.90349181 -0.17255841 -1.68369985]
{15, 20, 36, 39, 58, 73, 87, 98, 120, 123, 140, 169, 174, 181, 182, 203, 228, 232, 233, 243, 279, 294, 330, 335, 359, 367, 373, 402, 408, 417, 424, 441, 443, 458, 481, 482, 512, 536, 549, 553, 578, 583, 585, 609, 615, 621, 624, 638, 664, 669, 686, 696, 700, 702, 716, 721, 724, 728, 735, 759, 764, 770, 785, 786, 802, 804, 828, 854, 855, 857, 861, 882, 892, 899, 905, 908, 914, 957, 969, 977, 989, 995}
PHASE 3
[ 2.2533495   1.11272133 -1.45473268 -2.51865893 -1.08169041 -3.35924373
  1.50740204  1.1949467  -1.33670456 -0.65030632]
{120, 402, 583, 802, 899}
PHASE 4
[ 0.94939783  0.47762107 -0.18860086 -1.79436072  0.25816893 -2.11152462
  0.93971881  0.48616995 -1.66655475  0.85617959]
{120, 802, 899}
300
MAX_PHASE 4
PHASE 0
[-5.61515172  3.66766629 -4.94472677 -0.06129038 -1.71444693 -0.28918479
  8.65044438 -4.17682622  0.04344195 -1.89786612]
{0, 1, 2, 3, 4, 5, 6, 7

[-4.49032916 -5.97554168 -5.27400052  6.69339273  2.74828125  2.50253236
  1.67325765  6.48876781 -1.93016584 -1.05278075]
{15, 19, 23, 35, 38, 40, 43, 47, 56, 63, 68, 72, 74, 77, 87, 90, 100, 117, 124, 128, 132, 151, 157, 164, 176, 181, 184, 187, 198, 201, 210, 211, 212, 220, 225, 230, 239, 245, 246, 255, 260, 264, 276, 278, 281, 284, 285, 298, 303, 307, 310, 317, 320, 325, 327, 330, 331, 340, 342, 344, 345, 354, 355, 356, 359, 362, 363, 374, 380, 381, 386, 388, 392, 398, 400, 406, 414, 420, 421, 425, 433, 434, 435, 436, 437, 438, 441, 442, 446, 447, 452, 458, 461, 466, 472, 475, 476, 477, 478, 479, 485, 488, 497, 514, 515, 531, 534, 550, 553, 561, 571, 572, 575, 579, 590, 591, 598, 600, 605, 607, 612, 613, 623, 627, 629, 633, 643, 647, 654, 663, 669, 673, 676, 677, 679, 688, 691, 695, 708, 711, 712, 715, 717, 718, 721, 729, 735, 744, 747, 760, 776, 788, 790, 791, 802, 807, 813, 822, 830, 835, 837, 839, 844, 845, 846, 867, 879, 882, 888, 898, 901, 905, 907, 908, 918, 919, 926, 932, 93

...

[-1.1723677  -1.27811152  2.80692517 -2.00996094  1.37015934  6.10161415
  1.87837064  0.49326821 -0.50212711  1.53412481]
{0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 127, 128, 129, 130, 131, 132, 133, 134, 135, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200,

{2, 4, 5, 10, 23, 25, 31, 32, 48, 67, 82, 89, 91, 99, 101, 116, 131, 134, 135, 144, 157, 158, 164, 165, 170, 171, 173, 174, 177, 181, 191, 195, 209, 210, 222, 242, 249, 257, 259, 260, 263, 272, 303, 312, 314, 333, 348, 372, 401, 404, 437, 463, 470, 484, 500, 509, 519, 551, 558, 571, 572, 582, 619, 623, 636, 646, 650, 669, 679, 680, 691, 738, 774, 791, 809, 825, 829, 837, 843, 845, 848, 871, 886, 894, 896, 924, 940, 941, 945, 977}
PHASE 2
[-10.5780854   -1.96965078  -2.97692826   2.57350622   0.81378752
  -3.0722662    1.29303596  -4.53411808   4.2539708    4.66059318]
{2, 10, 25, 99, 134, 144, 157, 170, 171, 173, 181, 191, 195, 210, 257, 259, 260, 263, 272, 303, 312, 314, 333, 348, 372, 401, 437, 463, 509, 551, 571, 619, 623, 636, 646, 650, 669, 774, 791, 843, 845, 848, 894, 896, 924, 940, 945}
PHASE 3
[ 1.29830669 -3.20865777 -1.45121614 -0.01153389 -4.19718772  5.62392705
 -2.16275545 -5.93894187  3.35014944 -0.18880631]
{173, 348, 509, 945}
PHASE 4
[ 0.26339038 -0.06157859 -0.131976

[ 5.43775019 -8.46825545 -1.21974892 -0.05286089  9.46579066  7.15666044
 -5.86626468 16.30886527 -6.446809   -4.57360208]
{6, 17, 20, 26, 29, 33, 34, 36, 46, 52, 54, 62, 71, 76, 79, 88, 103, 109, 113, 114, 120, 122, 132, 134, 146, 147, 161, 165, 168, 169, 171, 172, 177, 185, 190, 193, 195, 200, 205, 210, 213, 227, 231, 237, 240, 254, 256, 258, 267, 276, 294, 296, 299, 315, 321, 325, 361, 362, 372, 377, 379, 392, 401, 402, 405, 412, 431, 440, 443, 456, 462, 471, 488, 491, 492, 505, 516, 531, 544, 546, 551, 560, 576, 581, 604, 613, 614, 620, 623, 631, 632, 633, 637, 652, 655, 658, 665, 686, 691, 729, 733, 739, 745, 755, 766, 771, 775, 777, 779, 780, 790, 827, 829, 834, 843, 851, 857, 868, 883, 886, 891, 894, 898, 899, 906, 908, 911, 914, 918, 919, 934, 936, 938, 940, 941, 951, 958, 978, 987, 989, 990, 992, 993, 995}
PHASE 2
[ 0.38899637  1.31735826 -0.56266675 -1.35191324  3.84472324 -5.14576312
  3.02649908  0.11779864 -0.64995357 -1.84404917]
{6, 20, 29, 33, 52, 54, 62, 76, 79, 88, 10

[-6.40462229 -3.38949947  1.5416521  -3.68760499 -3.6949966  -0.84725364
  1.18874714  1.03766528  6.85900169 -4.45032911]
{1, 2, 3, 4, 7, 8, 9, 10, 11, 13, 16, 17, 18, 21, 22, 23, 24, 25, 27, 28, 29, 30, 34, 36, 38, 39, 41, 43, 45, 46, 48, 49, 52, 56, 63, 65, 66, 68, 71, 72, 73, 74, 75, 78, 81, 82, 83, 85, 86, 87, 88, 89, 90, 92, 93, 94, 96, 97, 102, 104, 105, 108, 109, 111, 113, 117, 118, 120, 122, 123, 125, 126, 128, 130, 131, 133, 134, 136, 137, 139, 141, 142, 143, 146, 149, 150, 152, 155, 156, 157, 158, 163, 164, 165, 167, 168, 169, 173, 174, 176, 182, 185, 186, 190, 192, 194, 195, 196, 197, 199, 200, 201, 202, 206, 209, 210, 213, 214, 215, 216, 217, 218, 219, 220, 221, 223, 226, 227, 228, 229, 231, 233, 234, 238, 239, 240, 242, 244, 246, 247, 248, 249, 250, 252, 254, 256, 257, 259, 261, 264, 267, 268, 270, 272, 273, 275, 278, 281, 282, 283, 285, 287, 289, 291, 292, 294, 296, 297, 298, 299, 300, 301, 302, 304, 305, 308, 311, 312, 314, 315, 317, 319, 320, 324, 326, 327, 331, 334, 3

[  3.31326616  -2.69639288  -6.49338796   1.4015525    5.37903218
  -1.01275218 -14.43992882   9.50484881  -3.99448943  28.15775733]
{1, 11, 20, 37, 47, 48, 50, 55, 57, 66, 74, 86, 87, 95, 100, 112, 115, 153, 162, 171, 175, 176, 186, 193, 220, 227, 230, 234, 238, 241, 242, 258, 259, 277, 290, 295, 301, 312, 315, 334, 348, 358, 361, 384, 387, 393, 396, 412, 428, 446, 458, 460, 473, 474, 477, 478, 485, 486, 493, 517, 518, 521, 526, 535, 572, 592, 610, 621, 627, 642, 650, 659, 669, 673, 679, 680, 682, 687, 693, 698, 699, 707, 714, 719, 723, 728, 731, 733, 750, 751, 754, 758, 765, 771, 777, 787, 806, 812, 813, 822, 839, 857, 862, 866, 872, 882, 883, 921, 956, 958, 963, 978, 987, 994, 995}
PHASE 2
[ 1.71194099 -0.32786349 -1.28489869  1.62527596 -0.8732779  -0.5098575
  6.68216685  5.97908827  3.53373771 -4.54030705]
{1, 11, 20, 37, 47, 55, 57, 87, 95, 100, 153, 171, 186, 193, 227, 230, 234, 238, 242, 258, 259, 277, 295, 301, 312, 334, 348, 384, 393, 396, 412, 428, 458, 460, 473, 474, 485, 